# Начало тренировок

Для начала давайте разберем [тренировочный турнир](https://imcs.dvfu.ru/cats/problems?cid=6851588;sid=) от CODE work.

## Задача "Поиск в массиве"

Дана последовательность $a_1, a_2, \ldots, a_n$, для которой выполнено $a_i \leq a_{i+1}$ для любого $i=1..n-1$. Для каждого элемента последовательности $b_1, b_2, \ldots, b_m$ определить, присутствует ли он в последовательности $\{a_i\}$.

### Наивный алгоритм

In [1]:
# Находит число x в упорядоченном списке a
def find_in_sorted_container(a, x):
    return x in a

Ввод-вывод:

```python
with open('input.txt', 'r') as fin:
    n, m = map(int, fin.readline().split())
    a = list(map(int, fin.readline().split()))
    b = list(map(int, fin.readline().split()))

with open('output.txt', 'w') as fout:
    for x in b:
        if find_in_sorted_container(a, x):
            print("YES", file=fout)
        else:
            print("NO", file=fout)
```

Если отправить приведенное выше решение в тестирующую систему, то получится результат "Превышено время выполнения программы". Почему так происходит? В условии задачи сказано, что $n, m \leq 10^5$. Таким образом, цикл `for x in b` выполнится $m$ раз. Но функция `find_in_sorted_container`, несмотря на отсутствие явных циклов, потребует порядка $n$ обращений к элементам контейнера `a`. Следовательно, сложность наивного алгоритма составила $O(nm)$ операций.

Эта запись означает, что с ростом $n, m$ время выполнения вашей программы будет увеличиваться не сильнее, чем пропорционально $nm$. То есть, если 5 раз увеличить $n$ и $m$ в 10 раз, то оценка времени выполнения программы $O(nm)$ возрастет в $10^{10}$ раз - а это уже довольно много (для справки: одна секунда - это порядка $10^8$ - $10^9$ простейших операций).

### Решение с помощью структуры данных

Обратим внимание, как мы храним элементы массива `a`. В условии задачи сказано, что массив упорядочен по возрастанию, но мы никак этим не воспользовались. Вместо этого мы воспользовались операцией `in`, которая просматривает весь массив от начала до конца в поисках искомого числа.

Помимо массива, существуют другие способы организации хранения данных. Они различаются набором операций, выполняемых над множеством значений. В нашем случае нам нужна одна операция -- это поиск. Так что нам подойдет любая структура данных, обеспечивающая поиск в числовом множестве. Например, хеш-таблица. В языке Python такая структура данных уже реализована в стандартном контейнере `set`.

Давайте приведем наш список `a` к множеству `a_set` при помощи встроенной функции `set()`. А затем станем работать с этим новым контейнером.

```python
a_set = set(a)
# ...
    for x in b:
        if find_in_sorted_container(a_set, x):
# ...
```

Теперь наше решение принято в тестирующую систему. Однако с целью тренировки в программировании алгоритмов разберем ещё один способ.

### Двоичный поиск

Двоичный поиск - это самый эффективный алгоритм нахождения индекса заданного элемента в упорядоченном массиве. Алгоритм описан у [Кормена](https://yadi.sk/i/OgI3Y9bf5_NhAA). 

Идея алгоритма проста. Чтобы найти заданное число $x$ в последовательности $\{a_i\}$, которая представлена как массив (так что в ней быстро реализована операция доступа к элементу по его индексу), разобьем весь массив на два подмассива (почти) равной длины: $a[1..k]$ и $a[k+1..n]$. Элемент $a[k]$ будет *опорным*. Полагаем $k = \lfloor(l + r)/2\rfloor$, чтобы в худшем случае отбросить половину массива и уменьшить пространство поиска в 2 раза.

Справедливы утверждения:
* $x < a[k] \to \left(\forall i \in k+1..n \colon x < a[i] \right)$, то есть если $x$ меньше опорного элемента, то $x$ меньше любого элемента из правой половины массива;
* $x > a[k] \to \left(\forall i \in 1..k-1 \colon x > a[i] \right)$, то есть если $x$ больше опорного элемента, то $x$ больше любого элемента из левой половины массива.

Получаем алгоритм:

1. Если $x = a[k]$, то искомый элемент найден.
2. Если $x < a[k]$, то сводим задачу поиска в массиве $a[1..n]$ к задаче поиска в подмассиве $a[1..k-1]$.
3. Если $x > a[k]$, то сводим задачу поиска в массиве $a[1..n]$ к задаче поиска в подмассиве $a[k+1..n]$.

```{note}
При реализации данного алгоритма важно перед доступом к опорному элементу проверить, что подмассив непуст!
```

Время выполнения алгоритма составит $O(\log n)$, что гораздо быстрее, чем полный пере
бор, который требуе $O(n)$ операций.

Вариант рекурсивной реализации:

```python
def binary_search(a, x, l, r):
    if l > r:  # подмассив a[l..r] пуст?
        return False
    k = (l + r) // 2  # индекс опорного элемента
    if x == a[k]:
        return True
    elif x < a[k]:
        # ...
    else:  # x > a[k]
        # ...
        
def find_in_sorted_container(a, x):
    return binary_search(a, x, 0, n-1)
```


Вариант итеративной реализации:

```python
l = 0
r = n - 1
ans = False
while l <= r:
    # ищем число x в подмассиве a[l..r]
    k = (l + r) // 2
    if x == a[k]:
        ans = True
        break
    elif x < a[k]:
        # изменить l, r - свести задачу к другой
    else  # x > a[k]
        # изменить l, r - свести задачу к другой
# ans == True, если число x есть в массиве a[0..n-1]
```
.